# Data Exploration

+ In this notebook, we will explore the data from the IEA API.
+ We will load the data, explore it, decide what to keep and create a function to load the data.


In [12]:
import pandas as pd

In [13]:
ev_sales_url = "https://api.iea.org/evs?parameters=EV%20sales&category=Historical&mode=Cars&csv=true"

ev_charging_points_url = "https://api.iea.org/evs?parameters=EV%20charging%20points&category=Historical&mode=EV&csv=true"

ev_sales_df = pd.read_csv(ev_sales_url)
ev_charging_points_df = pd.read_csv(ev_charging_points_url)

display(ev_sales_df.head())
display(ev_charging_points_df.head())

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.0
1,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,440.0
2,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.0
3,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,670.0
4,Australia,Historical,EV charging points,EV,Publicly available slow,2019,charging points,1700.0


In [14]:
ev_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      3798 non-null   object 
 1   category    3798 non-null   object 
 2   parameter   3798 non-null   object 
 3   mode        3798 non-null   object 
 4   powertrain  3798 non-null   object 
 5   year        3798 non-null   int64  
 6   unit        3798 non-null   object 
 7   value       3798 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 237.5+ KB


In [15]:
ev_sales_df[ev_sales_df['parameter'] == 'EV sales'].head()

,region,category,parameter,mode,powertrain,year,unit,value
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
5,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
9,Australia,Historical,EV sales,Cars,PHEV,2012,Vehicles,80.0
10,Australia,Historical,EV sales,Cars,PHEV,2013,Vehicles,100.0
14,Australia,Historical,EV sales,Cars,BEV,2013,Vehicles,190.0


In [16]:
ev_sales_df[ev_sales_df['parameter'] == 'EV sales share'].head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.0065
7,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.0300
12,Australia,Historical,EV sales share,Cars,EV,2013,percent,0.0340
19,Australia,Historical,EV sales share,Cars,EV,2014,percent,0.1600
24,Australia,Historical,EV sales share,Cars,EV,2015,percent,0.2000


In [17]:
for column in ev_sales_df.columns:
    if ev_sales_df[column].dtype != 'float64':
        print(column)
        print(ev_sales_df[column].unique())
        print("")


region
['Australia' 'Austria' 'Belgium' 'Brazil' 'Bulgaria' 'Canada' 'Chile'
 'China' 'Colombia' 'Costa Rica' 'Croatia' 'Cyprus' 'Czech Republic'
 'Denmark' 'Estonia' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece'
 'Hungary' 'Iceland' 'India' 'Ireland' 'Israel' 'Italy' 'Japan' 'Korea'
 'Latvia' 'Lithuania' 'Luxembourg' 'Mexico' 'Netherlands' 'New Zealand'
 'Norway' 'Poland' 'Portugal' 'Rest of the world' 'Romania' 'Seychelles'
 'Slovakia' 'Slovenia' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Turkiye' 'United Arab Emirates' 'United Kingdom' 'USA' 'World']

category
['Historical']

parameter
['EV sales share' 'EV stock share' 'EV sales' 'EV stock'
 'Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge']

mode
['Cars']

powertrain
['EV' 'BEV' 'PHEV' 'FCEV']

year
[2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2010]

unit
['percent' 'Vehicles' 'GWh' 'Milion barrels per day'
 'Oil displacement, million lge']



Let's drop the columns:
+ `mode`
+ `category`
+ `parameter`

In [18]:
# Split the df into ev_sales_df, ev_charging_points_df
ev_sales_df = ev_sales_df[ev_sales_df['parameter'] == 'EV sales']
ev_charging_points_df = ev_charging_points_df[ev_charging_points_df['parameter'] == 'EV charging points']

# Drop the columns
cols_to_drop = ['mode', 'category', 'parameter']
ev_sales_df = ev_sales_df.drop(columns=cols_to_drop)
ev_charging_points_df = ev_charging_points_df.drop(columns=cols_to_drop)

# Rename the columns
ev_sales_df = ev_sales_df.rename(columns={"value": "Cars Sold", "region": "Region", "year": "Year", "powertrain": "Powertrain"})
ev_charging_points_df = ev_charging_points_df.rename(columns={"value": "Charging Points", "region": "Region", "year": "Year", "powertrain": "Powertrain"})

## load_data()

In [19]:
def load_data():
    """
    Load the data from the IEA API
    """

    # IEA API URLs
    ev_sales_url = "https://api.iea.org/evs?parameters=EV%20sales&category=Historical&mode=Cars&csv=true"
    ev_charging_points_url = "https://api.iea.org/evs?parameters=EV%20charging%20points&category=Historical&mode=EV&csv=true"

    # Load the data from the IEA API
    ev_sales_df = pd.read_csv(ev_sales_url)
    ev_charging_points_df = pd.read_csv(ev_charging_points_url)

    # Split the df into ev_sales_df, ev_charging_points_df
    ev_sales_df = ev_sales_df[ev_sales_df['parameter'] == 'EV sales']
    ev_charging_points_df = ev_charging_points_df[ev_charging_points_df['parameter'] == 'EV charging points']

    # Drop the columns
    cols_to_drop = ['mode', 'category', 'parameter']
    ev_sales_df = ev_sales_df.drop(columns=cols_to_drop)
    ev_charging_points_df = ev_charging_points_df.drop(columns=cols_to_drop)

    # Rename the columns
    ev_sales_df = ev_sales_df.rename(columns={"value": "Cars Sold", "region": "Region", "year": "Year", "powertrain": "Powertrain"})
    ev_charging_points_df = ev_charging_points_df.rename(columns={"value": "Charging Points", "region": "Region", "year": "Year", "powertrain": "Powertrain"})

    print("Data loaded successfully")

    return ev_sales_df, ev_charging_points_df


In [20]:
ev_sales_df, ev_sales_share_df, ev_charging_points_df = load_data()

Data loaded successfully


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
ev_charging_points_df.head()

## top_regions_by_ev_sales()

In [ ]:
ev_sales_df, ev_sales_share_df, ev_charging_points_df = load_data()

In [ ]:

regions_to_remove = ['World', 'EU27', 'Europe', 'World']

ev_sales_df = ev_sales_df[~ev_sales_df["Region"].isin(regions_to_remove)]

ev_sales_df = ev_sales_df[ev_sales_df["Year"] == ev_sales_df["Year"].max()]

# Add up the values for each powertrain, reset index turns the Series into a DataFrame
ev_sales_region_df = ev_sales_df.groupby(["Region"])["Cars Sold"].sum().reset_index()
# Sort the values
ev_sales_region_df = ev_sales_region_df.sort_values(by="Cars Sold", ascending=False)

# Change data type to int
ev_sales_region_df["Cars Sold"] = ev_sales_region_df["Cars Sold"].astype(int)

# Reset index
ev_sales_region_df = ev_sales_region_df.reset_index(drop=True).head(10)

ev_sales_region_df


In [13]:
def top_regions_by_ev_sales(ev_sales_df, year):
    """
    Get the top 10 regions by EV sales
    """

    # Remove the regions that we don't want to include
    regions_to_remove = ['World', 'EU27', 'Europe', 'World']
    ev_sales_df = ev_sales_df[~ev_sales_df["Region"].isin(regions_to_remove)]

    # Filter the data for the selected year
    ev_sales_df = ev_sales_df[ev_sales_df["Year"] == year]

    # Add up the values for each powertrain, reset index turns the Series into a DataFrame
    ev_sales_region_df = ev_sales_df.groupby(["Region"])["Cars Sold"].sum().reset_index()
    # Sort the values
    ev_sales_region_df = ev_sales_region_df.sort_values(by="Cars Sold", ascending=False)

    # Change data type to int
    ev_sales_region_df["Cars Sold"] = ev_sales_region_df["Cars Sold"].astype(int)

    # Reset index
    ev_sales_region_df = ev_sales_region_df.reset_index(drop=True).head(10)

    return ev_sales_region_df

In [ ]:
df1 = top_regions_by_ev_sales(ev_sales_df, 2023)
df1

## world_ev_sales_metric_card()

In [ ]:
ev_sales_df, ev_sales_share_df, ev_charging_points_df = load_data()

In [ ]:
ev_sales_df.tail(15)

In [17]:
total_sales_current_year = ev_sales_df[ev_sales_df["Region"] == "World"]

current_year = ev_sales_df["Year"].max()
previous_year = current_year - 1

total_sales_previous_year = total_sales_current_year.groupby("Year")["Cars Sold"].sum().loc[previous_year]
total_sales_current_year = total_sales_current_year.groupby("Year")["Cars Sold"].sum().loc[current_year]

# Convert to millions
total_sales_previous_year = total_sales_previous_year / 1_000_000
total_sales_current_year = total_sales_current_year / 1_000_000

In [32]:
# Create a function that returns the last two years of data for the total EV sales metric card
def world_ev_sales_metric_card(ev_sales_df, year):
    world_ev_sales_df = ev_sales_df[ev_sales_df["Region"] == "World"]

    current_year = year
    previous_year = current_year - 1

    world_ev_sales_previous_year = world_ev_sales_df.groupby("Year")["Cars Sold"].sum().loc[previous_year]
    world_ev_sales_current_year = world_ev_sales_df.groupby("Year")["Cars Sold"].sum().loc[current_year]

    delta_previous_year = world_ev_sales_current_year - world_ev_sales_previous_year

    # Convert to millions
    world_ev_sales_current_year = world_ev_sales_current_year / 1_000_000
    delta_previous_year = delta_previous_year / 1_000_000

    return world_ev_sales_current_year, delta_previous_year

In [ ]:
current_year, delta_previous_year = world_ev_sales_metric_card(ev_sales_df, 2023)
current_year, delta_previous_year


## world_charging_points_metric_card()

In [ ]:
ev_charging_points_df

In [ ]:
world_charging_points_df = ev_charging_points_df[ev_charging_points_df["Region"] == "World"]

current_year = world_charging_points_df["Year"].max()
previous_year = current_year - 1    

world_charging_points_previous_year = world_charging_points_df.groupby("Year")["Charging Points"].sum().loc[previous_year]
world_charging_points_current_year = world_charging_points_df.groupby("Year")["Charging Points"].sum().loc[current_year]

delta_previous_year = world_charging_points_current_year - world_charging_points_previous_year

world_charging_points_current_year, delta_previous_year

In [29]:
def world_charging_points_metric_card(ev_charging_points_df):
    """
    Get the total EV charging points for the current and previous year
    """

    # Filter the data for the World region
    world_charging_points_df = ev_charging_points_df[ev_charging_points_df["Region"] == "World"]

    # Get the current and previous year
    current_year = world_charging_points_df["Year"].max()
    previous_year = current_year - 1    

    # Get the total charging points for the current and previous year
    world_charging_points_previous_year = world_charging_points_df.groupby("Year")["Charging Points"].sum().loc[previous_year]
    world_charging_points_current_year = world_charging_points_df.groupby("Year")["Charging Points"].sum().loc[current_year]

    # Calculate the delta between the current and previous year
    delta_previous_year = world_charging_points_current_year - world_charging_points_previous_year

    # Convert to millions
    world_charging_points_current_year = world_charging_points_current_year / 1_000_000
    delta_previous_year = delta_previous_year / 1_000_000

    return world_charging_points_current_year, delta_previous_year


In [ ]:
current_year, delta_previous_year = world_charging_points_metric_card(ev_charging_points_df)
current_year, delta_previous_year

## world_ev_sales_growth_metric_card()

In [38]:
def world_ev_sales_growth_metric_card(ev_sales_df, year):
    """
    Get the current and previous year of data for the World EV Sales Growth metric card
    """

    # Filter the data for the World region
    world_ev_sales_df = ev_sales_df[ev_sales_df["Region"] == "World"]

    # Get the current and previous year
    current_year = year
    previous_year = current_year - 1
    previous_year_2 = previous_year - 1

    # Get the total sales for the current and previous year
    world_ev_sales_previous_year = world_ev_sales_df.groupby("Year")["Cars Sold"].sum().loc[previous_year]
    world_ev_sales_current_year = world_ev_sales_df.groupby("Year")["Cars Sold"].sum().loc[current_year]
    world_ev_sales_previous_year_2 = world_ev_sales_df.groupby("Year")["Cars Sold"].sum().loc[previous_year_2]

    # Current year growth
    current_year_growth = (world_ev_sales_current_year / world_ev_sales_previous_year) - 1
    # Previous year growth
    previous_year_growth = (world_ev_sales_previous_year / world_ev_sales_previous_year_2) - 1

    return current_year_growth, previous_year_growth

In [ ]:
current_year_growth, previous_year_growth = world_ev_sales_growth_metric_card(ev_sales_df, 2023)
current_year_growth, previous_year_growth